In [39]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

In [13]:
# Load your data
df = pd.read_csv("/Users/gavinwentzel/Desktop/Grad School/Mirabito Project/Mirabito_Documentation/Final_Data_Long.csv")


In [45]:
df = df.dropna(subset=["Loyalty_Program"])

# FP-Growth function with top 20 and readable formatting
def run_fpgrowth_on_subset(subset_df, min_support=0.01):
    grouped = subset_df.groupby('Transaction_ID')['Item_Category'].apply(list).tolist()
    te = TransactionEncoder()
    te_ary = te.fit(grouped).transform(grouped)
    basket_df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = fpgrowth(basket_df, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
    rules = rules.sort_values(by="lift", ascending=False).head(20)

    # Convert frozenset to readable strings
    rules["antecedents"] = rules["antecedents"].apply(lambda x: ", ".join(sorted(x)))
    rules["consequents"] = rules["consequents"].apply(lambda x: ", ".join(sorted(x)))
    rules["Rule"] = rules["antecedents"] + " → " + rules["consequents"]
    return rules

# Define labeled segments
all_rules = []

segments = {
    "Mirabito Loyalty": df[df['Loyalty_Program'] == 'Mirabito Loyalty'],
    "Alt ID Loyalty": df[df['Loyalty_Program'] == 'Mirabito Alt ID'],
    "Rewards and Payment": df[df['Loyalty_Program'] == 'Rewards and Payment'],
    "High CLV": df[df['CLV_Tier'] == 'High'],
    "Fuel Transactions": df[df['Transaction_ID'].isin(df[df['Item_Category'] == 'FUEL']['Transaction_ID'].unique()) &
                            (df['Item_Category'] != 'FUEL')]
}

# Generate rules for each segment
for label, subset in segments.items():
    rules = run_fpgrowth_on_subset(subset)
    rules["Source"] = label
    all_rules.append(rules)

# Combine and export
combined_rules_df = pd.concat(all_rules, ignore_index=True)
combined_rules_df.to_csv("/Users/gavinwentzel/Desktop/basketAnalysis.csv", index=False)

print("Exported to Desktop as: basketAnalysis.csv")

Exported to Desktop as: basketAnalysis.csv


In [37]:
# 3 Questions CSVs

import pandas as pd
import itertools

# Load your data
df = pd.read_csv("/Users/gavinwentzel/Desktop/Final_Data_Long.csv")

# 1. Average fuel quantity by loyalty group
avg_fuel_by_loyalty = df[df["Item_Category"] == "FUEL"].groupby("Loyalty_Program")["Quantity"].mean().reset_index()
avg_fuel_by_loyalty.columns = ["Loyalty_Program", "Average_Gallons"]
avg_fuel_by_loyalty.to_csv("avg_fuel_by_loyalty.csv", index=False)

# 2. Co-purchased item pairs
grouped_tx = df.groupby("Transaction_ID")["Item_Category"].apply(list)
pair_counts = {}

for items in grouped_tx:
    unique_items = list(set(items))
    for pair in itertools.combinations(sorted(unique_items), 2):
        pair_counts[pair] = pair_counts.get(pair, 0) + 1

pair_df = pd.DataFrame([
    {"Item_A": k[0], "Item_B": k[1], "Count": v} for k, v in pair_counts.items()
])
pair_df.to_csv("item_pair_counts.csv", index=False)

# 3. Transaction-level basket table
basket_table = df.groupby(['Transaction_ID', 'Customer_PK', 'Loyalty_Program', 'CLV_Tier'])['Item_Category'] \
    .apply(lambda x: ', '.join(sorted(set(x)))).reset_index()
basket_table.columns = ['Transaction_ID', 'Customer_PK', 'Loyalty_Program', 'CLV_Tier', 'Basket_Items']
basket_table.to_csv("transaction_baskets.csv", index=False)

print("CSVs exported: avg_fuel_by_loyalty.csv, item_pair_counts.csv, transaction_baskets.csv")

       Transaction_ID  Customer_PK Loyalty_Program   ItemID  Quantity  \
1504          2489764       607834    Weis Loyalty  3275984      1.00   
1505          2489764       607834    Weis Loyalty  3273458      1.00   
25119         1900267       759975    Weis Loyalty       -1     11.02   
25120         1900267       759975    Weis Loyalty       -1     11.02   
25121         1900267       759975    Weis Loyalty       -1      1.00   

          Transaction_Date Transaction_Time      CLV CLV_Tier  \
1504   2025-05-28 00:00:00         11:00:00    8.000      Low   
1505   2025-05-28 00:00:00         11:00:00    8.000      Low   
25119  2025-05-28 00:00:00         16:30:00  973.184     High   
25120  2025-05-28 00:00:00         16:30:00  973.184     High   
25121  2025-05-28 00:00:00         16:30:00  973.184     High   

           Item_Category  Unit_Price  Item_Revenue  
1504   Cig Promotions(R)        0.25          0.25  
1505          CIGARETTES       15.69         15.69  
25119      

In [ ]:
# CLV frequent bundles
import pandas as pd
import itertools
from collections import Counter

# Load your dataset
df = pd.read_csv("/Users/gavinwentzel/Desktop/Final_Data_Long.csv")

# Filter for rows with a valid CLV Tier
clv_grouped_df = df[df["CLV_Tier"].notna()]

# Function to compute pair metrics per CLV_Tier
def compute_grouped_pair_metrics(filtered_df, group_label):
    results = []
    for group_val, group_df in filtered_df.groupby(group_label):
        total_tx = group_df["Transaction_ID"].nunique()
        tx_series = group_df.groupby("Transaction_ID")["Item_Category"].apply(set)

        item_ct = Counter()
        pair_ct = Counter()
        for items in tx_series:
            item_ct.update(items)
            for pair in itertools.combinations(sorted(items), 2):
                pair_ct[pair] += 1

        for (a, b), count in pair_ct.items():
            support = count / total_tx
            confidence_a_to_b = count / item_ct[a]
            confidence_b_to_a = count / item_ct[b]
            lift_a_to_b = confidence_a_to_b / (item_ct[b] / total_tx)
            lift_b_to_a = confidence_b_to_a / (item_ct[a] / total_tx)
            results.append({
                group_label: group_val,
                "Item_A": a,
                "Item_B": b,
                "Pair_Count": count,
                "Support": round(support, 4),
                "Confidence_A_to_B": round(confidence_a_to_b, 4),
                "Confidence_B_to_A": round(confidence_b_to_a, 4),
                "Lift_A_to_B": round(lift_a_to_b, 2),
                "Lift_B_to_A": round(lift_b_to_a, 2),
                "Item_Pair": f"{a} → {b}"
            })
    return pd.DataFrame(results)

# Compute all pair metrics for CLV_Tier groups
pair_metrics_by_clv = compute_grouped_pair_metrics(clv_grouped_df, "CLV_Tier")

# Rank by support within each CLV_Tier and filter top 5
pair_metrics_by_clv["Support_Rank"] = pair_metrics_by_clv.groupby("CLV_Tier")["Support"].rank(method="first", ascending=False)
top_clv_by_support = pair_metrics_by_clv[pair_metrics_by_clv["Support_Rank"] <= 5].sort_values(["CLV_Tier", "Support_Rank"])

# Export to CSV
top_clv_by_support.to_csv("frequent_bundles_CLV.csv", index=False)

print("File saved: frequent_bundles_CLV.csv")

In [47]:
# Group items by transaction
transactions = df.groupby("Transaction_ID")["Item_Category"].apply(list).tolist()

# One-hot encode items
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
basket_df = pd.DataFrame(te_ary, columns=te.columns_)

# Run FP-Growth
frequent_itemsets = fpgrowth(basket_df, min_support=0.01, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Clean up the rules for readability
rules["antecedents"] = rules["antecedents"].apply(lambda x: ", ".join(sorted(x)))
rules["consequents"] = rules["consequents"].apply(lambda x: ", ".join(sorted(x)))
rules["Rule"] = rules["antecedents"] + " → " + rules["consequents"]

# Sort and keep top 100 by lift
rules = rules.sort_values(by="lift", ascending=False).head(100)

# Export to Desktop
rules.to_csv("/Users/gavinwentzel/Desktop/basket_analysis.csv", index=False)

print("Exported to Desktop as: basket_analysis.csv")

✅ Exported to Desktop as: basket_analysis.csv
